# Sentiment Classification on a Multilingual Tweet Dataset

This project focuses on fine-tuning the multilingual language model [XLM-RoBERTa-base](https://huggingface.co/FacebookAI/xlm-roberta-base) on a sentiment classification task using [multilingual tweets](https://huggingface.co/FacebookAI/xlm-roberta-base) as data. The dataset contains tweets labeled with sentiment categories in several languages.

# Install Libraries

In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install bertviz transformers
!pip install accelerate --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 11.7 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.3.0
    Uninstalling accelerate-1.3.0:
      Successfully uninstalled accelerate-1.3.0


In [ ]:
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
import torch
import numpy as np
import evaluate

# Load and Preprocess the Dataset

The Tweet Sentiment Multilingual dataset contains tweets labeled as **negative**, **neutral**, and **positive**. To make the training efficient, a truncation limit is set so a maximum of 100 words to prevent unneccessary memory usage.

The dataset is divided into a training, validation, and test set.

In [ ]:
# Load the multilingual tweet sentiment dataset
tweet_dataset = load_dataset("cardiffnlp/tweet_sentiment_multilingual", 'all')

# Function to truncate text for efficiency
def truncate(example):
    return {
        'text': " ".join(example['text'].split()[:100]),
        'label': example['label']
    }

# Reduce dataset size
small_tweet_dataset = DatasetDict(
    train=tweet_dataset['train'].shuffle(seed=24).select(range(5000)).map(truncate),
    val=tweet_dataset['validation'].shuffle(seed=24).select(range(1000)).map(truncate),
    test=tweet_dataset['test'].shuffle(seed=24).select(range(1000)).map(truncate),
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.28k [00:00<?, ?B/s]

tweet_sentiment_multilingual.py:   0%|          | 0.00/4.14k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/187k [00:00<?, ?B/s]

...

Generating validation split:   0%|          | 0/2592 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6960 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

# Tokenization

During the tokenization the raw text is converted into numerical values, needed for the model to process them. The XLM-RoBERTa tokenizer supports multiple languages which is important for this specific task.

In [ ]:
# Load tokenizer
model_name = "FacebookAI/xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [ ]:
# Keep text before tokenization
def tokenize_function(examples):
    tokenized_examples = tokenizer_xlm_roberta(examples["text"], padding=True, truncation=True)
    tokenized_examples["text"] = examples["text"]
    return tokenized_examples

# Apply tokenization and keep text
small_tokenized_dataset = small_tweet_dataset.map(tokenize_function, batched=True, batch_size=16)
small_tokenized_dataset = small_tokenized_dataset.rename_column("label", "labels")
small_tokenized_dataset.set_format("torch")


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

# Load the Model

In [ ]:
# Define data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Load pre-trained multilingual model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3) # 3 sentiment classes

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Training

In [ ]:
# Load evaluation metric
accuracy = evaluate.load("accuracy")

# Define training arguments
training_args = TrainingArguments(
    output_dir="mydrive/tweet_trainer",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=50,
    num_train_epochs=5,
    learning_rate=2e-5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to='none',
    seed=224
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_tokenized_dataset['train'],
    eval_dataset=small_tokenized_dataset['val'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

<ipython-input-8-5a3698b6e764>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Train model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.000200,0.881311,0.600000
2,0.823500,0.837071,0.615000
3,0.730600,0.882735,0.618000
4,0.646700,0.891189,0.630000
5,0.571700,0.914382,0.632000


TrainOutput(global_step=785, training_loss=0.7710945761127836, metrics={'train_runtime': 6632.4709, 'train_samples_per_second': 3.769, 'train_steps_per_second': 0.118, 'total_flos': 1042081246647504.0, 'train_loss': 0.7710945761127836, 'epoch': 5.0})

In [ ]:
# What Checkpoint is what Epoch
'''
1	checkpoint-157
2	checkpoint-314
3	checkpoint-471
4	checkpoint-628
5	checkpoint-785
'''

'\n1\tcheckpoint-157\n2\tcheckpoint-314\n3\tcheckpoint-471\n4\tcheckpoint-628\n5\tcheckpoint-785\n'

# Evaluation

In [ ]:
# Evaluate the model
trainer.evaluate()

{'eval_loss': 0.837071418762207,
 'eval_accuracy': 0.615,
 'eval_runtime': 59.9937,
 'eval_samples_per_second': 16.668,
 'eval_steps_per_second': 0.533,
 'epoch': 5.0}

# Continue Training

In [ ]:
# Load the best checkpoint
best_checkpoint = "mydrive/tweet_trainer/checkpoint-785"
model = AutoModelForSequenceClassification.from_pretrained(best_checkpoint)

training_args = TrainingArguments(
    output_dir="mydrive/tweet_trainer2",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=50,
    num_train_epochs=10, # Increased
    learning_rate=2e-5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to='none',
    seed=224
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_tokenized_dataset["train"],
    eval_dataset=small_tokenized_dataset["val"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Resume training
trainer.train(resume_from_checkpoint="mydrive/tweet_trainer/checkpoint-785")



<ipython-input-16-b97af65cdd6e>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/transformers/trainer.py:3441: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where 

Epoch,Training Loss,Validation Loss,Accuracy
6,0.540400,0.927784,0.637000
7,0.485300,0.994446,0.624000
8,0.428600,1.060001,0.628000
9,0.374800,1.093254,0.634000
10,0.348200,1.091038,0.638000


TrainOutput(global_step=1570, training_loss=0.22631755361131803, metrics={'train_runtime': 7526.1206, 'train_samples_per_second': 6.644, 'train_steps_per_second': 0.209, 'total_flos': 2069251362574704.0, 'train_loss': 0.22631755361131803, 'epoch': 10.0})

In [ ]:
# What Checkpoint is what Epoch
'''
6	checkpoint-942
7	checkpoint-1099
8	checkpoint-1256
9	checkpoint-1413
10	checkpoint-1570
'''

'\n6\tcheckpoint-942\n7\tcheckpoint-1099\n8\tcheckpoint-1256\n9\tcheckpoint-1413\n10\tcheckpoint-1570\n'

# Evaluation

In [ ]:
# Evaluate the model
trainer.evaluate()

{'eval_loss': 0.837071418762207,
 'eval_accuracy': 0.615,
 'eval_runtime': 45.7675,
 'eval_samples_per_second': 21.85,
 'eval_steps_per_second': 0.699,
 'epoch': 10.0}

In [ ]:
# I tried several manually created sentence in this cell
test_text = "I drive a blue car."
model_inputs = tokenizer(test_text, return_tensors="pt")
prediction = torch.argmax(model(**model_inputs).logits)
print(["Negative", "Neutral", "Positive"][prediction])

Neutral


# Evaluate the Model on the test set

In [ ]:
# Load the best checkpoint for TEST
best_checkpoint = "mydrive/tweet_trainer2/checkpoint-1413"
model = AutoModelForSequenceClassification.from_pretrained(best_checkpoint)

training_args = TrainingArguments(
    output_dir="mydrive/tweet_trainer2",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=50,
    num_train_epochs=10,
    learning_rate=2e-5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to='none',
    seed=224
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_tokenized_dataset["train"],
    eval_dataset=small_tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

<ipython-input-32-d58459f60f4b>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Evaluate the trained model on the test set
test_results = trainer.evaluate(eval_dataset=small_tokenized_dataset["test"])
print(test_results)

{'eval_loss': 1.0152422189712524, 'eval_model_preparation_time': 0.0039, 'eval_accuracy': 0.645, 'eval_runtime': 57.3381, 'eval_samples_per_second': 17.44, 'eval_steps_per_second': 0.558}


In [ ]:
# Save the final model
model.save_pretrained("drive/MyDrive/tweet_sentiment_model")
tokenizer.save_pretrained("drive/MyDrive/tweet_sentiment_model")

('drive/MyDrive/tweet_sentiment_model/tokenizer_config.json',
 'drive/MyDrive/tweet_sentiment_model/special_tokens_map.json',
 'drive/MyDrive/tweet_sentiment_model/sentencepiece.bpe.model',
 'drive/MyDrive/tweet_sentiment_model/added_tokens.json',
 'drive/MyDrive/tweet_sentiment_model/tokenizer.json')

In [ ]:
# More manually created example sentences
test_sentences = [
    "I love this product!",
    "The service was terrible.",
    "It was okay, nothing special.",
    "I use the car."
]

# Run predictions
for text in test_sentences:
    model_inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        logits = model(**model_inputs).logits
    prediction = torch.argmax(logits).item()

    print(f"Text: {text}")
    print(f"Predicted Sentiment: {['Negative', 'Neutral', 'Positive'][prediction]}")

Text: I love this product!
Predicted Sentiment: Positive
Text: The service was terrible.
Predicted Sentiment: Negative
Text: It was okay, nothing special.
Predicted Sentiment: Positive
Text: I use the car.
Predicted Sentiment: Neutral


Visualization for Epoch 10

In [ ]:
import os
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

# Load Model and Tokenizer
model_path = "/content/drive/MyDrive/tweet_sentiment_model"
checkpoint_path = "/content/drive/MyDrive/tweet_trainer2/checkpoint-1570" # Load epoch 10

fine_tuned_model_xlm_roberta = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer_xlm_roberta = AutoTokenizer.from_pretrained(model_path)

# Prepare test dataset
num_examples = 500
small_test_dataset = small_tokenized_dataset["test"].shuffle(seed=42).select(range(num_examples))

sentence_list = small_test_dataset["text"]
labels_list = small_test_dataset["labels"]

# Tokenize inputs
model_inputs = tokenizer_xlm_roberta(
    sentence_list,
    padding=True, truncation=True, return_tensors='pt'
)

# Run inference to get hidden states
fine_tuned_model_xlm_roberta.eval()
with torch.no_grad():
    outputs = fine_tuned_model_xlm_roberta(**model_inputs, output_hidden_states=True)

# Define path for visualization results
path = "drive/MyDrive/results_visualization"
os.makedirs(path, exist_ok=True)

all_hidden_states = outputs.hidden_states
num_layers = len(all_hidden_states)

# Iterate over layers
for layer in range(num_layers):
    layer_dir = os.path.join(path, f"layer_{layer}")
    os.makedirs(layer_dir, exist_ok=True)

    tensors = []
    labels = []

    for example in range(all_hidden_states[layer].shape[0]):
        cls_embedding = all_hidden_states[layer][example][0]  # CLS Token
        tensors.append(cls_embedding)

        label = [
            sentence_list[example],
            str(labels_list[example])
        ]
        labels.append(label)

    # Save embeddings
    embeddings_tensor = torch.stack(tensors)
    writer = SummaryWriter(log_dir=layer_dir)
    writer.add_embedding(embeddings_tensor, metadata=labels, metadata_header=['text', 'label'])
    writer.close()

# Visualization for Epoch 5

In [ ]:
import os
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

# Load Model and Tokenizer
model_path = "/content/drive/MyDrive/tweet_sentiment_model"
checkpoint_path = "/content/drive/MyDrive/tweet_trainer/checkpoint-785" # Load epoch 5

fine_tuned_model_xlm_roberta = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer_xlm_roberta = AutoTokenizer.from_pretrained(model_path)

# Prepare test dataset
num_examples = 500
small_test_dataset = small_tokenized_dataset["test"].shuffle(seed=42).select(range(num_examples))

sentence_list = small_test_dataset["text"]
labels_list = small_test_dataset["labels"]

# Tokenize inputs
model_inputs = tokenizer_xlm_roberta(
    sentence_list,
    padding=True, truncation=True, return_tensors='pt'
)

# Run inference to get hidden states
fine_tuned_model_xlm_roberta.eval()
with torch.no_grad():
    outputs = fine_tuned_model_xlm_roberta(**model_inputs, output_hidden_states=True)

# Define path for visualization results
path = "drive/MyDrive/results_visualization_epoch5"
os.makedirs(path, exist_ok=True)

all_hidden_states = outputs.hidden_states
num_layers = len(all_hidden_states)

# Iterate over layers
for layer in range(num_layers):
    layer_dir = os.path.join(path, f"layer_{layer}")
    os.makedirs(layer_dir, exist_ok=True)

    tensors = []
    labels = []

    for example in range(all_hidden_states[layer].shape[0]):
        cls_embedding = all_hidden_states[layer][example][0]  # CLS Token
        tensors.append(cls_embedding)

        label = [
            sentence_list[example],
            str(labels_list[example])
        ]
        labels.append(label)

    # Save embeddings
    embeddings_tensor = torch.stack(tensors)
    writer = SummaryWriter(log_dir=layer_dir)
    writer.add_embedding(embeddings_tensor, metadata=labels, metadata_header=['text', 'label'])
    writer.close()